In [1]:
from funkwpap import sympy_to_gurobi, Firm, Country, Sector, Regulator, get_emission
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys
from gurobipy import Model, LinExpr, QuadExpr, GRB
x = sp.symbols('x')
y = sp.symbols('y')

In [2]:

# Create Sectors and Countries
Regulator1 = Regulator('EU', permit_price= 2, emission_cap= 100)
sector1 = Sector('cement', price_demand_function= 100 - 0.1*x, free_emission_multiplier= 0.1, regulator= Regulator1)
sector2 = Sector('steel', price_demand_function=150 - 0.1*x, free_emission_multiplier= 0.05, regulator= Regulator1)
sector3 = Sector('paper', price_demand_function= 200 - 0.02*x**1.5, free_emission_multiplier= 0.1, regulator= Regulator1)
country1 = Country('DE', 1, regulator= Regulator1)
country2 = Country('FI', 0.5, regulator= Regulator1)
country3 = Country('GR', size= 0.1, regulator= Regulator1)

# Create Firms using objects
firm1 = Firm('firm1', 1, 1, x*0, 0.1*sp.exp('x') + 100/y**2, 0, 0, 0, regulator= Regulator1)
firm2 = Firm('firm2', 1, 2, x*0, 5*x**3.2 + 40/y**2, 0, 0, 0, regulator= Regulator1)
firm3 = Firm('firm3', 1, 3, x*0, x**5.2 + 30/y**2, 0, 0, 0, regulator= Regulator1)
firm4 = Firm('firm4', 2, 1, x*0, x**3.3 + 20/y**2, 0, 0, 0, regulator= Regulator1)
firm5 = Firm('firm5', 2, 2, x*0, x**2.3 + 20/y**2, 0, 0, 0, regulator= Regulator1)
firm6 = Firm('firm6', 2, 3, x*0, x**3.2 + 15/y**2, 0, 0, 0, regulator= Regulator1)
firm7 = Firm('firm7', 3, 1, x*0, x**4.1 + 40/y**2, 0, 0, 0, regulator= Regulator1)
firm8 = Firm('firm8', 3, 2, x*0, x**5.1 + 50/y**2, 0, 0, 0, regulator= Regulator1)
firm9 = Firm('firm9', 3, 3, x*0, x**5 + 60/y**2, 0, 0, 0, regulator= Regulator1)


print(sector1.firms)  # List of firms in sector1
print(country1.firms)  # List of firms in country1
print(Regulator1.firm_registry)  # Dictionary of all firms

for firm in Regulator1.firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name)


[Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=2, name='firm2', sector_id=1, country_id=2, actual_output=0, emission=0, profit=0), Firm(id=3, name='firm3', sector_id=1, country_id=3, actual_output=0, emission=0, profit=0)]
[Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=4, name='firm4', sector_id=2, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=7, name='firm7', sector_id=3, country_id=1, actual_output=0, emission=0, profit=0)]
{1: Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), 2: Firm(id=2, name='firm2', sector_id=1, country_id=2, actual_output=0, emission=0, profit=0), 3: Firm(id=3, name='firm3', sector_id=1, country_id=3, actual_output=0, emission=0, profit=0), 4: Firm(id=4, name='firm4', sector_id=2, country_id=1, actual_output=0, emission=0, profit=0), 5: Firm(id=5, name='firm5', sector_id=2, country_id=2, actual_out

In [3]:
#Regulator1.optimize_them_all(precision = 0.1, print_diff = True, BAU = True)

In [4]:
for firm in Regulator1.firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name, firm.actual_output, firm.emission, firm.profit)

firm1 cement DE 0 0 0
firm2 cement FI 0 0 0
firm3 cement GR 0 0 0
firm4 steel DE 0 0 0
firm5 steel FI 0 0 0
firm6 steel GR 0 0 0
firm7 paper DE 0 0 0
firm8 paper FI 0 0 0
firm9 paper GR 0 0 0


In [5]:
Regulator1.BAU_calculator(print_diff = True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-14
Max diff: 0.01, permit price = 2.0, cap = 100, second_stage = False, a = 1.0000

In [6]:
print("Firm name, Sector, Country, Actual output, Emission, Profit, BAU profit, BAU emission")
for firm in Regulator1.firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name, firm.actual_output, firm.emission, firm.profit, firm.BAU_profit, firm.BAU_emission)

Firm name, Sector, Country, Actual output, Emission, Profit, BAU profit, BAU emission
firm1 cement DE 257.32525385607016 0.0 6273.59076591368 6273.59076591368 257.32525385607016
firm2 cement FI 257.32525385607016 0.0 6273.59076591368 6273.59076591368 257.32525385607016
firm3 cement GR 241.54945043799546 0.0 5888.97661255950 5888.97661255950 241.54945043799546
firm4 steel DE 376.13222749306016 0.0 13977.1983562917 13977.1983562917 376.13222749306016
firm5 steel FI 376.13222749306016 0.0 13977.1983562917 13977.1983562917 376.13222749306016
firm6 steel GR 376.13222749306016 0.0 13977.1983562917 13977.1983562917 376.13222749306016
firm7 paper DE 118.62743152590413 0.0 7800.22830116322 7800.22830116322 118.62743152590413
firm8 paper FI 118.57196583476036 0.0 7796.62338066782 7796.62338066782 118.57196583476036
firm9 paper GR 118.6398990833682 0.0 7800.69946375400 7800.69946375400 118.6398990833682


In [7]:
# Create a plot for the permit price with respect to the emission cap 10 samples between 0 and 200
BAU_percentage = np.linspace(0, 1, 20)
BAU_emissions = Regulator1.BAU_emissions
caps = [BAU_emissions*i for i in BAU_percentage]
# Read the data from the csv file, if it exists, if not, just create a new one
try:
    df = pd.read_csv('permit_price_to_Percentage_of_BAU_v0_0.csv')
    percentages = list(df['Percentage of BAU'])
    caps2 = list(df['Emission Cap'])
    permit_prices = list(df['Permit Price'])
    outputs = list(df['Outputs'])
    emissions = list(df['Emissions'])
    profits = list(df['Profits'])
except:
    percentages = []
    caps2 = []
    permit_prices = []
    outputs = []
    emissions = []
    profits = []
# Loop through all the emission caps and calculate the permit price

# Find all the caps that are not calculated yet (meaning that they are more than 0.5 different than a value of the caps2)
for i in caps[:]:  # Iterate over a copy of the list
    if any(abs(i - x) < 0.5 for x in caps2):
        caps.remove(i)

for i in tqdm.tqdm(caps):
    Regulator1.emission_cap = i  
    Regulator1.find_optimal_permit_price_to_meet_the_emission_cap_requirements(precision= 0.01, x_high = 1000000000000)
    caps2.append(i)
    percentages.append(Regulator1.emission_cap/BAU_emissions)
    permit_prices.append(Regulator1.permit_price)
    outputs.append(tuple([firm.actual_output for firm in Regulator1.firm_registry.values()]))
    emissions.append(tuple([firm.emission for firm in Regulator1.firm_registry.values()]))
    profits.append(tuple([firm.profit for firm in Regulator1.firm_registry.values()]))
    # save all the data in a csv file
    df = pd.DataFrame({'Percentage of BAU': percentages, 'Emission Cap': caps2, 'Permit Price': permit_prices, 'Outputs': outputs, 'Emissions': emissions, 'Profits': profits})
    df.to_csv('permit_price_to_Percentage_of_BAU_v0_0.csv', index=False)




0it [00:00, ?it/s]


Εδώ θα τρέξει το ίδιο, αλ΄λα κοιτώντας μόνο το τελευταίο για να δει αν αποκλίνει. 

In [8]:
# Create a plot for the permit price with respect to the emission cap 10 samples between 0 and 200
BAU_percentage = np.linspace(0, 1, 20)
BAU_emissions = Regulator1.BAU_emissions
caps = [BAU_emissions*i for i in BAU_percentage]
# Read the data from the csv file, if it exists, if not, just create a new one
try:
    df = pd.read_csv('permit_price_to_Percentage_of_BAU_v1_0.csv')
    percentages = list(df['Percentage of BAU'])
    caps2 = list(df['Emission Cap'])
    permit_prices = list(df['Permit Price'])
    outputs = list(df['Outputs'])
    emissions = list(df['Emissions'])
    profits = list(df['Profits'])
except:
    percentages = []
    caps2 = []
    permit_prices = []
    outputs = []
    emissions = []
    profits = []
# Loop through all the emission caps and calculate the permit price

# Find all the caps that are not calculated yet (meaning that they are more than 0.5 different than a value of the caps2)
for i in caps[:]:  # Iterate over a copy of the list
    if any(abs(i - x) < 0.5 for x in caps2):
        caps.remove(i)

for i in tqdm.tqdm(caps):
    Regulator1.emission_cap = i  
    Regulator1.find_optimal_permit_price_to_meet_the_emission_cap_requirements(precision= 0.01, x_high = 1000000000000, size_of_diffs= 1)
    caps2.append(i)
    percentages.append(Regulator1.emission_cap/BAU_emissions)
    permit_prices.append(Regulator1.permit_price)
    outputs.append(tuple([firm.actual_output for firm in Regulator1.firm_registry.values()]))
    emissions.append(tuple([firm.emission for firm in Regulator1.firm_registry.values()]))
    profits.append(tuple([firm.profit for firm in Regulator1.firm_registry.values()]))
    # save all the data in a csv file
    df = pd.DataFrame({'Percentage of BAU': percentages, 'Emission Cap': caps2, 'Permit Price': permit_prices, 'Outputs': outputs, 'Emissions': emissions, 'Profits': profits})
    df.to_csv('permit_price_to_Percentage_of_BAU_v1_0.csv', index=False)




  0%|          | 0/20 [00:00<?, ?it/s]

Max diff: 0.00, permit price = 1000000000000.0, cap = 0, second_stage = False, a = 1.000

  5%|▌         | 1/20 [01:06<21:03, 66.48s/it]

Permit price: 1000000000000 and total emission: 0.008999990999988938 and emission cap 0.0
Max diff: 0.12, permit price = 203.7, cap = 118, second_stage = True, a = 0.011000000000Everything failed, best result is 0.010177164748505163
Permit price: 179.625204rice = 179.6, cap = 118, second_stage = False, a = 0.430

 10%|█         | 2/20 [22:53<3:58:50, 796.11s/it]

...

Permit price: 117.32481652870774 and total emission: 471.1759908188006 and emission cap 471.67072359438924
Max diff: 0.65, permit price = 107.3, cap = 590, second_stage = True, a = 0.01003000000000Everything failed, best result is 0.041190852145518875
Max diff: 0.01, permit price = 106.4, cap = 590, second_stage = False, a = 1.000

 30%|███       | 6/20 [2:13:11<6:09:42, 1584.44s/it]

Permit price: 106.4108801074326 and total emission: 578.7277995743306 and emission cap 589.5884044929866
Max diff: 0.27, permit price = 94.6, cap = 708, second_stage = True, a = 0.011000000000000Everything failed, best result is 0.03032923625761441
Permit price: 98.680175price = 98.7, cap = 708, second_stage = False, a = 0.483

 35%|███▌      | 7/20 [2:44:44<6:05:10, 1685.42s/it]

Permit price: 98.68017514236271 and total emission: 698.7054534270387 and emission cap 707.5060853915838
Max diff: 0.07, permit price = 116.4, cap = 825, second_stage = True, a = 0.17100000000000